In [ ]:
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.io import ascii
from glob import glob
from astropy.table import Table
import matplotlib.pyplot as plt

In [ ]:
# MAD > 5 times MAG error


<h3>Classes and definitions</h3>

In [ ]:
#######################################
### Read from additioned FLT frames ###
#######################################

class read_sexcats():
    def __init__(self):
        #phot_df = self.join(1)
        self.phot_df = self.join()#2, phot_df)
        #phot_df = self.return_func(phot_df)
        
    def join(self, phot_df=None):
        items = np.sort(glob('../Add_Regrid/*add_regrid.fits'))
        #np.sort([w.split('_flt')[0] for w in glob('./wfc'+str(wfc)+'/*rate*regrid.fits')])
        for iterator in range(len(items)):
            item = items[iterator]
            hdul = fits.open(item)
            expstart = hdul[1].header['EXPSTART']
            EXPTIME  = hdul[1].header['EXPTIME']
            filter_  = hdul[1].header['FILTER']
            sexcat_file = item.replace('add_regrid', 'phot')
            sexcat = Table.read(sexcat_file, format="fits", hdu='LDAC_OBJECTS')#.to_pandas()
            sexcat = self.unpack_apers(sexcat)
            sexcat.remove_columns(['MAG_APER', 'MAGERR_APER', 'FLUX_APER', 'FLUXERR_APER'])    
            sexcat = sexcat.to_pandas()

            sexcat = sexcat.rename(columns={'NUMBER':'SExID'})
            sexcat = sexcat.rename(columns={'VECTOR_ASSOC':'AssocID'})
            
            sexcat['AssocID'] = sexcat['AssocID'].astype(int)
            sexcat = sexcat[sexcat.AssocID!=0]

            sexcat['T_Start'] = expstart
            sexcat['Exptime'] = EXPTIME
            sexcat['Exp_Length'] = ('deep' if EXPTIME > 31 else 'short')
            sexcat['Filter']     = filter_
            #sexcat['WhichWFC']   = wfc
            sexcat = self.clean(sexcat)
            #sexcat = sexcat.set_index('ID')
            try:
                phot_df = pd.concat((phot_df,sexcat), axis=0)
            except:
                phot_df = sexcat
        return phot_df
        
    def return_func(self):
        print('Done. Returning photometry dataframe')
        return self.phot_df.set_index(['AssocID', 'Filter', 'T_Start', 'Exp_Length']).sort_index()
    
    def clean(self, sexcat):
        """I take all rules from the HCV catalogue"""
        sexcat = sexcat[(sexcat.MAG_APER2<80)&(sexcat.MAG_AUTO<80)]
        sexcat = sexcat[sexcat.MAG_APER2<31.0]
        sexcat = sexcat[sexcat.MAG_AUTO<35.0]
        sexcat = sexcat[sexcat.FLAGS<=7]
        sexcat = sexcat[sexcat.MAGERR_APER2<0.2]
        return sexcat
    
    def unpack_apers(self, sexcat):
        """Unpacking the stacked lists
           NOTES:
           APER1 and APER2 refer to Whitmore et al 2016 (2.5 and 7.5 pixels diameter)
           APER3 refers to our own aperture, which is 6 pixels in diameter
           """
        sexcat['FLUX_APER1'] = [w[0] for w in sexcat['FLUX_APER']]
        sexcat['FLUX_APER3'] = [w[1] for w in sexcat['FLUX_APER']]
        sexcat['FLUX_APER2'] = [w[2] for w in sexcat['FLUX_APER']]
        sexcat['FLUXERR_APER1'] = [w[0] for w in sexcat['FLUXERR_APER']]
        sexcat['FLUXERR_APER3'] = [w[1] for w in sexcat['FLUXERR_APER']]
        sexcat['FLUXERR_APER2'] = [w[2] for w in sexcat['FLUXERR_APER']]

        sexcat['MAG_APER1'] = [w[0] for w in sexcat['MAG_APER']]
        sexcat['MAG_APER3'] = [w[1] for w in sexcat['MAG_APER']]
        sexcat['MAG_APER2'] = [w[2] for w in sexcat['MAG_APER']]
        sexcat['MAGERR_APER1'] = [w[0] for w in sexcat['MAGERR_APER']]
        sexcat['MAGERR_APER3'] = [w[1] for w in sexcat['MAGERR_APER']]
        sexcat['MAGERR_APER2'] = [w[2] for w in sexcat['MAGERR_APER']]
        return sexcat

In [ ]:
#############################################
### Read from single regridded FLT frames ###
#############################################
class read_sexcats():
    def __init__(self):
        phot_df = self.join(1)
        self.phot_df = self.join(2, phot_df)
        #phot_df = self.return_func(phot_df)
        
    def join(self, wfc, phot_df=None):
        items = np.sort([w.split('_flt')[0] for w in glob('./wfc'+str(wfc)+'/*rate*regrid.fits')])
        for iterator in range(len(items)):
            item = items[iterator]
            hdul = fits.open(glob(item+'*regrid*')[0])
            expstart = hdul[0].header['EXPSTART']
            EXPTIME  = hdul[0].header['EXPTIME']
            filter_  = hdul[0].header['FILTER']
            sexcat = Table.read(glob(item+'*phot*')[0], format="fits", hdu='LDAC_OBJECTS')#.to_pandas()
            sexcat = self.unpack_apers(sexcat)
            sexcat.remove_columns(['MAG_APER', 'MAGERR_APER', 'FLUX_APER', 'FLUXERR_APER'])    
            sexcat = sexcat.to_pandas()

            sexcat = sexcat.rename(columns={'NUMBER':'SExID'})
            sexcat = sexcat.rename(columns={'VECTOR_ASSOC':'AssocID'})
            
            sexcat['AssocID'] = sexcat['AssocID'].astype(int)
            sexcat = sexcat[sexcat.AssocID!=0]
            if wfc==2: 
                sexcat.SExID += phot_df[phot_df.WhichWFC==1].SExID.max()
            #sexcat = sexcat.rename(columns={'VECTOR_ASSOC':'ID'})
            #sexcat['ID'] = sexcat['ID'].astype(int)

            sexcat['T_Start'] = expstart
            sexcat['Exptime'] = EXPTIME
            sexcat['Exp_Length'] = ('deep' if EXPTIME > 31 else 'short')
            sexcat['Filter']     = filter_
            sexcat['WhichWFC']   = wfc
            sexcat = self.clean(sexcat)
            #sexcat = sexcat.set_index('ID')
            try:
                phot_df = pd.concat((phot_df,sexcat), axis=0)
            except:
                phot_df = sexcat
        return phot_df
        
    def return_func(self):
        print('Done. Returning photometry dataframe')
        return self.phot_df.set_index(['AssocID', 'Filter', 'T_Start']).sort_index()
    
    def clean(self, sexcat):
        """I take all rules from the HCV catalogue"""
        sexcat = sexcat[(sexcat.MAG_APER2<80)&(sexcat.MAG_AUTO<80)]
        sexcat = sexcat[sexcat.MAG_APER2<31.0]
        sexcat = sexcat[sexcat.MAG_AUTO<35.0]
        sexcat = sexcat[sexcat.FLAGS<=7]
        sexcat = sexcat[sexcat.MAGERR_APER2<0.2]
        return sexcat
    
    def unpack_apers(self, sexcat):
        """Unpacking the stacked lists
           NOTES:
           APER1 and APER2 refer to Whitmore et al 2016 (2.5 and 7.5 pixels diameter)
           APER3 refers to our own aperture, which is 6 pixels in diameter
           """
        sexcat['FLUX_APER1'] = [w[0] for w in sexcat['FLUX_APER']]
        sexcat['FLUX_APER3'] = [w[1] for w in sexcat['FLUX_APER']]
        sexcat['FLUX_APER2'] = [w[2] for w in sexcat['FLUX_APER']]
        sexcat['FLUXERR_APER1'] = [w[0] for w in sexcat['FLUXERR_APER']]
        sexcat['FLUXERR_APER3'] = [w[1] for w in sexcat['FLUXERR_APER']]
        sexcat['FLUXERR_APER2'] = [w[2] for w in sexcat['FLUXERR_APER']]

        sexcat['MAG_APER1'] = [w[0] for w in sexcat['MAG_APER']]
        sexcat['MAG_APER3'] = [w[1] for w in sexcat['MAG_APER']]
        sexcat['MAG_APER2'] = [w[2] for w in sexcat['MAG_APER']]
        sexcat['MAGERR_APER1'] = [w[0] for w in sexcat['MAGERR_APER']]
        sexcat['MAGERR_APER3'] = [w[1] for w in sexcat['MAGERR_APER']]
        sexcat['MAGERR_APER2'] = [w[2] for w in sexcat['MAGERR_APER']]
        return sexcat

In [ ]:
################################################################
### Read from unaltered FLT frames with regridded whitelights###
################################################################
class read_sexcats():
    def __init__(self):
        phot_df = self.join(1)
        self.phot_df = self.join(2, phot_df)
        #phot_df = self.return_func(phot_df)
        
    def join(self, wfc, phot_df=None):
        items = np.sort([w.split('_flt')[0] for w in glob('../SingleFrame_DetRegrid/WFC'+str(wfc)+'/*phot.fits')])
        items = items
        for iterator in range(len(items)):
            item = items[iterator]
            hdul = fits.open(glob(item+'*pamcorr_rate*')[0])
            expstart = hdul[0].header['EXPSTART']
            EXPTIME  = hdul[0].header['EXPTIME']
            if EXPTIME < 31:
                continue
            filter_  = hdul[0].header['FILTER']
            sexcat = Table.read(glob(item+'*phot*')[0], format="fits", hdu='LDAC_OBJECTS')#.to_pandas()
            sexcat = self.unpack_apers(sexcat)
            sexcat.remove_columns(['MAG_APER', 'MAGERR_APER', 'FLUX_APER', 'FLUXERR_APER'])    
            sexcat = sexcat.to_pandas()

            sexcat = sexcat.rename(columns={'NUMBER':'SExID'})
            sexcat = sexcat.rename(columns={'VECTOR_ASSOC':'AssocID'})
            #sexcat['AssocID'] = 1
            
            sexcat['AssocID'] = sexcat['AssocID'].astype(int)
            sexcat = sexcat[sexcat.AssocID!=0]
            if wfc==2: 
                sexcat.SExID += phot_df[phot_df.WhichWFC==1].SExID.max()
            #sexcat = sexcat.rename(columns={'VECTOR_ASSOC':'ID'})
            #sexcat['ID'] = sexcat['ID'].astype(int)

            sexcat['T_Start'] = expstart
            sexcat['Exptime'] = EXPTIME
            sexcat['Exp_Length'] = ('deep' if EXPTIME > 31 else 'short')
            sexcat['Filter']     = filter_
            sexcat['WhichWFC']   = wfc
            sexcat = self.clean(sexcat)
            #sexcat = sexcat.set_index('ID')
            try:
                phot_df = pd.concat((phot_df,sexcat), axis=0)
            except:
                phot_df = sexcat
        return phot_df
        
    def return_func(self):
        print('Done. Returning photometry dataframe')
        return self.phot_df.set_index(['AssocID', 'Filter', 'T_Start']).sort_index()
    
    def clean(self, sexcat):
        """I take all rules from the HCV catalogue"""
        sexcat = sexcat[(sexcat.MAG_APER2<80)&(sexcat.MAG_AUTO<80)]
        sexcat = sexcat[sexcat.MAG_APER2<31.0]
        sexcat = sexcat[sexcat.MAG_AUTO<35.0]
        sexcat = sexcat[sexcat.FLAGS<=7]
        sexcat = sexcat[sexcat.MAGERR_APER2<0.2]
        return sexcat
    
    def unpack_apers(self, sexcat):
        """Unpacking the stacked lists
           NOTES:
           APER1 and APER2 refer to Whitmore et al 2016 (2.5 and 7.5 pixels diameter)
           APER3 refers to our own aperture, which is 6 pixels in diameter
           """
        sexcat['FLUX_APER1'] = [w[0] for w in sexcat['FLUX_APER']]
        sexcat['FLUX_APER3'] = [w[1] for w in sexcat['FLUX_APER']]
        sexcat['FLUX_APER2'] = [w[2] for w in sexcat['FLUX_APER']]
        sexcat['FLUXERR_APER1'] = [w[0] for w in sexcat['FLUXERR_APER']]
        sexcat['FLUXERR_APER3'] = [w[1] for w in sexcat['FLUXERR_APER']]
        sexcat['FLUXERR_APER2'] = [w[2] for w in sexcat['FLUXERR_APER']]

        sexcat['MAG_APER1'] = [w[0] for w in sexcat['MAG_APER']]
        sexcat['MAG_APER3'] = [w[1] for w in sexcat['MAG_APER']]
        sexcat['MAG_APER2'] = [w[2] for w in sexcat['MAG_APER']]
        sexcat['MAGERR_APER1'] = [w[0] for w in sexcat['MAGERR_APER']]
        sexcat['MAGERR_APER3'] = [w[1] for w in sexcat['MAGERR_APER']]
        sexcat['MAGERR_APER2'] = [w[2] for w in sexcat['MAGERR_APER']]
        return sexcat


In [ ]:
class HCV_error_analysis():
    def __init__(self, phot_df):
        self.phot_df = phot_df
        return

    def get_CI(self,phot_df):
        CI = phot_df['MAG_APER1'] - phot_df['MAG_APER2']
        phot_df['CI'] = CI
        return phot_df

    def get_synthetic_err(self,phot_df):
        """Following the approach of the HCV paper"""
        # I take the median, but note that there is always only ONE count, thus median = value
        MagerrAper2  = phot_df.groupby(['AssocID', 'Filter','T_Start'])['MAGERR_APER2'].median()
        MagerrAper2_timemedian = phot_df.groupby(['AssocID', 'Filter'])['MAGERR_APER2'].median()
        MagDivision = (MagerrAper2 / MagerrAper2_timemedian)**2
        
        CI_timemedian = phot_df.CI.groupby(['AssocID', 'Filter']).median()
        CIDivision    = (phot_df.CI / CI_timemedian)**2
        
        MagAper_MagAuto = (phot_df.MAG_APER2 - phot_df.MAG_AUTO)
        MagAper_MagAuto_timemedian = MagAper_MagAuto.groupby(['AssocID', 'Filter']).median()
        MagAper_MagAutoDivision = (MagAper_MagAuto/MagAper_MagAuto_timemedian)**2
        
        RATimemedian = phot_df.groupby(['AssocID', 'Filter', 'T_Start'])['ALPHA_J2000'].median()
        DECTimemedian = phot_df.groupby(['AssocID', 'Filter', 'T_Start'])['DELTA_J2000'].median()

        RAs = phot_df.groupby(['AssocID','Filter'])['ALPHA_J2000'].median()
        DECs = phot_df.groupby(['AssocID', 'Filter'])['DELTA_J2000'].median()

        DeltaRAs = np.power(RAs - RATimemedian,2)
        DeltaDECs = np.power(DECs - DECTimemedian,2)

        Dists = np.sqrt(DeltaRAs+DeltaDECs)
        DistsTimemedian = Dists.groupby(['AssocID', 'Filter']).median()
        DistsDivision   = (Dists/DistsTimemedian)**2
        
        SynError = np.sqrt(MagDivision + CIDivision + MagAper_MagAutoDivision + DistsDivision)
        #ID = phot_df.groupby('SExID')['VECTOR_ASSOC'].median().astype(int)
        #ID = ID.apply(lambda x: 1000000+x)
        #ID[ID==1000000] = pd.Series(ID.index).apply(lambda x: 2000000+x).values
        phot_df['SynError'] = SynError
        
        return phot_df
    
    def linfit(self,x,b):
        return 1/np.inf*x+b
    
    def get_sigma_prime(self, phot_df):
        StarIDs = phot_df.get_level_values(0).unique()
        for StarID in StarIDs:
            filters = phot_df.loc[StarIDs].index.get_level_values(0).unique()
    
    
    
    def get_synthetic_err_deviation(self, phot_df):
        SynErrorSTD = phot_df.groupby(['AssocID', 'Filter'])['SynError'].std() # Standard deviation of synthetic errors
        SynErrorMed = phot_df.groupby(['AssocID', 'Filter'])['SynError'].median() # median of synthetic errors
        SynErrorDev = (phot_df.SynError-SynErrorMed).abs()/SynErrorSTD # How many sigma is this measurement removed from the median
        phot_df['SynErrorDevSig'] = SynErrorDev.values
        phot_df['SynErrorMed'] = SynErrorMed
        phot_df['SynErrorSig'] = phot_df.SynError / SynErrorMed
        phot_df['High_sigma_syn'] = phot_df['SynErrorSig'] > 4
        return phot_df
    
    
    def robust_mean(self,mags, synerrs):
        mags = mags.values
        synerrs = synerrs.values
        synerrs = synerrs[np.argsort(mags)]
        mags = np.sort(mags)
        if len(mags)>5:
            to_drop = (len(mags)//6)
            mags = mags[to_drop:-to_drop]
            synerrs = synerrs[to_drop:-to_drop]
        weighted_mean = np.nansum(1/synerrs**2 * mags) / np.nansum(1/synerrs**2)
        return weighted_mean, np.sqrt(np.mean(np.power(mags-weighted_mean,2)))
    
    def get_mean_sigmaprime(self, phot_df):
        phot_df = self.phot_df.copy()
        """This will take some time"""
        StarIDs = phot_df.index.get_level_values(0).unique().values
        phot_df['RobustMean'] = 0
        phot_df['RobustSigma'] = 0
        i=0
        for StarID in StarIDs:
            filters = phot_df.loc[StarID].index.get_level_values(0).unique()
            for this_filter in filters:
                sub_phot_df = phot_df.loc[StarID,this_filter]
                Lightcurve_SynErrors = sub_phot_df.SynError
                Lightcurve_mags = sub_phot_df.MAG_APER2
                TrueMag, SigmaPrime = self.robust_mean(Lightcurve_mags, Lightcurve_SynErrors)
                phot_df.loc[(StarID, this_filter,),'RobustMean'] = TrueMag
                phot_df.loc[(StarID, this_filter,),'RobustSigma']= SigmaPrime
        phot_df['SigmaPrimeDeviation'] = (phot_df['MAG_APER2'] - phot_df['RobustMean']).abs() / phot_df['RobustSigma']
        phot_df['High_sigma_prime'] = phot_df['SigmaPrimeDeviation'] > 4
        return phot_df
    
    def get_local_mzp_corr(self, phot_df):
        phot_df['MagDiff'] = phot_df['MAG_APER2'] - phot_df['RobustMean']
        nearest = nearest_neighbours(phot_df)
        phot_df = phot_df[phot_df.Exp_Length == 'deep'].copy()
        
        IDs = phot_df.index.get_level_values(0).unique()
        phot_df['MZPCorr']=0
        for ID in IDs:
            neightbours = nearest.loc[ID]
            photometry = phot_df.loc[neightbours.values]
            MagOffset = photometry.groupby(['Filter', 'T_Start'])['MagDiff'].median()
            MagOffset = pd.DataFrame(MagOffset)
            MagOffset.columns = ['MZPCorr2']
            phot_df.loc[ID,'MZPCorr'] = pd.DataFrame(phot_df.loc[ID]).join(MagOffset)['MZPCorr2'].values
        phot_df['MAG_APER2_ORIG'] = phot_df['MAG_APER2']
        phot_df['MAG_APER2'] = phot_df['MAG_APER2'] - phot_df['MZPCorr']
        return phot_df
    
    
    def clean(self, MZPcorr=False):
        phot_df = self.get_CI(self.phot_df)
        phot_df = self.get_synthetic_err(phot_df)
        phot_df = self.get_synthetic_err_deviation(phot_df)
        phot_df = self.get_mean_sigmaprime(phot_df)
        if MZPcorr:
            phot_df = self.get_local_mzp_corr(phot_df)
        phot_df = phot_df[phot_df.CI < 5]
        phot_df = phot_df[(np.logical_not(phot_df.High_sigma_prime))*(np.logical_not(phot_df.High_sigma_syn))]
        print('Done. Returning photometry dataframe including CI and synthetic errors and MZP correction')
        return phot_df


In [ ]:
def SExToCat(phot_df, cat):
    SExCoordinates = phot_df.groupby('AssocID')['ALPHA_J2000', 'DELTA_J2000'].median()
    CatCoordinates = all_stars[['RA', 'Dec']]
    DeltaRA  = np.power(SExCoordinates.ALPHA_J2000.values - CatCoordinates.RA.values[:,np.newaxis],2, dtype=np.float32).T
    DeltaDec = np.power(SExCoordinates.DELTA_J2000.values - CatCoordinates.Dec.values[:,np.newaxis],2, dtype=np.float32).T
    Dists    = np.sqrt(DeltaRA + DeltaDec)
    Dists[Dists>1e-4] = np.nan
    del DeltaRA, DeltaDec
    Dists     = pd.DataFrame(Dists, index = SExCoordinates.index, columns=all_stars.index)
    Assoc_df  = pd.DataFrame({'RefCatID':Dists.idxmin(axis=1)})
    return Assoc_df.dropna()
def nearest_neighbours(phot_df, Nstars = 50):
    catalogue = phot_df.groupby('AssocID')['ALPHA_J2000', 'DELTA_J2000'].median()
    Nentries = len(catalogue)
    
    deltaRA  = np.memmap('deltaRA.temp' , dtype='float64', mode='w+', shape=(Nentries,Nentries))
    deltaDEC = np.memmap('deltaDec.temp', dtype='float64', mode='w+', shape=(Nentries,Nentries))
    
    deltaRA[:]  = catalogue.ALPHA_J2000.values - catalogue.ALPHA_J2000.values[:,np.newaxis]
    deltaDEC[:] = catalogue.DELTA_J2000.values - catalogue.DELTA_J2000.values[:,np.newaxis]
    
    deltaAngle = deltaRA**2 + deltaDEC**2
    del deltaRA, deltaDEC
    
    nearest = catalogue.index.values[np.argsort(deltaAngle, axis=1)] # To correct for ID <-> argument
    nearest = pd.DataFrame(nearest[:,1:Nstars+1], index=catalogue.index)
    return nearest

In [ ]:
class Analysis():
    def __init__(self, phot_df):
        self.ApplySelection(phot_df)
        return

    def ApplySelection(self, phot_df):
        self.phot_df = phot_df[phot_df.SynErrorDevSig<=3]
        self.phot_df = self.phot_df[self.phot_df.Exp_Length=='deep']
    
    def GetMADvalues(self):
        phot_df          = self.phot_df
        MedianMagTime    = phot_df.groupby(['AssocID', 'Filter', 'T_Start'])['MAG_APER2'].median()
        MedianMagOverall = phot_df.groupby(['AssocID', 'Filter'])['RobustMean'].median()
        MagOffset        = (MedianMagTime - MedianMagOverall).abs()
        self.MAD         = MagOffset.groupby(['AssocID', 'Filter']).median()
        return self.MAD
    
    def GetMedMags(self):
        self.MedMag = self.phot_df.groupby(['AssocID', 'Filter'])['MAG_APER2'].median()
        return self.MedMag
    
    def GetMADPlot(self, filter_=None):
        self.GetMADvalues()
        self.MAD.hist(bins=np.arange(0,1,0.002),cumulative=False, histtype='step', normed=False, linewidth=2)
        plt.xlabel('MAD Value')
        plt.ylabel('PDF')
        plt.show()
        
        self.MAD.hist(bins=np.arange(0,1,0.002),cumulative=True, histtype='step', normed=True, linewidth=2)
        plt.xlabel('MAD Value')
        plt.ylabel('CDF')
        plt.show()
        
        self.GetMedMags()
        if filter_ !=None:
            med_mags = self.MedMag[self.MedMag.index.get_level_values(1)==filter_]
            mad_vals = self.MAD[self.MAD.index.get_level_values(1)==filter_]
        else:
            med_mags = self.MedMag#[self.MedMag.index.get_level_values(1)==filter_]
            mad_vals = self.MAD#[self.MAD.index.get_level_values(1)==filter_]
        plt.hist2d(med_mags, mad_vals, bins=(np.linspace(15,28,80), np.linspace(0,0.4,50)), cmap=plt.cm.jet)
        plt.xlabel('Median magnitude')
        plt.ylabel('MAG Value')
        plt.show()
        
    
    def DrawTrumpets(self,magcol = 'MAG_APER3', IDcol = 'AssocID'):
        phot_df = self.phot_df
        merrcol = magcol.replace('MAG', 'MERR')
        # Trumpet plot
        from matplotlib.colors import LogNorm
        magnitudes = phot_df[magcol]
        filters = magnitudes.index.get_level_values(1).unique()
        plt.figure(figsize=(20,10))
        num=0
        for filter_ in np.sort(filters):
            epochs  = magnitudes.loc[:,filter_,:].index.get_level_values(1).unique()
            mags_t0 = pd.DataFrame(magnitudes.loc[:, filter_, epochs[0], :])
            num+=1
            plt.subplot('23'+str(num))
            xs, ys = np.array([]), np.array([])
            for i in range(len(epochs)-1):
                mags_t1 = pd.DataFrame(magnitudes.loc[:, filter_, epochs[i+1], :])
                join_mags = mags_t0.join(mags_t1, lsuffix = '_t0', rsuffix='_t1', how='inner')
                join_mags['DeltaMag'] = join_mags[magcol+'_t1'] - join_mags[magcol+'_t0']
                median_mags = magnitudes.loc[:, filter_, :].groupby([IDcol]).median()
                join_mags = join_mags.join(median_mags)
                x, y = join_mags[magcol].values, join_mags['DeltaMag'].values
                x, y = x[np.isfinite(x)*np.isfinite(y)], y[np.isfinite(x)*np.isfinite(y)]
                xs = np.hstack((xs,x))
                ys = np.hstack((ys,y))
            plt.hist2d(xs, ys, bins=(np.linspace(15,28,80), np.linspace(-3,3,150)), cmap='jet', cmin=15)
            plt.title(filter_, pad=-15)
            plt.ylabel('Delta Mag')
            plt.xlabel('Median magnitude')
            plt.ylim(-1,1)
            plt.xlim(14,27)
            if num in [1,4,7]:
                plt.ylabel('Delta Mag')
            else:
                plt.yticks([])
            if num in [4,5]:
                plt.xlabel('Median magnitude')
            else:
                plt.xticks([])
            plt.ylim(-1.5,1.5)
        plt.tight_layout()
        #plt.savefig('trumpet_diagrams.png', dpi=500)
        plt.show()

    def MAG_pdf(self, magcol = 'MAG_APER2',fluxcol='FLUX_APER2', IDcol = 'AssocID'):
        phot_df = self.phot_df
        fig, ax = plt.subplots(figsize=(10,6))
        for name, group in phot_df[phot_df[fluxcol]>0].groupby('Filter')[magcol]:
            group.hist(bins=np.arange(15,30,0.1), histtype='step', ax=ax, normed=True, label=name, linewidth=2)
        ax.legend()
        ax.set_xlabel('Magnitude')
        ax.set_ylabel('Normalized frequency')
        ax.set_xlim(15,30)
        plt.show()


        fig, ax = plt.subplots(figsize=(10,6))
        for name, group in phot_df[phot_df[fluxcol]>0].groupby('Filter')[magcol]:
            group.hist(bins=np.arange(15,30,0.1),cumulative=True, histtype='step', ax=ax, normed=True, label=name, linewidth=2)
        ax.legend()
        ax.set_xlabel('Magnitude')
        ax.set_ylabel('Normalized frequency')
        ax.set_xlim(15,30)
        plt.show()
    
    def GetNumberOfMeasurements(self):
        counts = self.phot_df['MAG_APER2'].groupby(['AssocID', 'Filter']).count()
        return counts
    
    def GetTimeSeries(self, AssocID = None, Filter=None):
        if not AssocID:
            MinMeasurements = 10
            WhichEntries    = self.GetNumberOfMeasurements()>MinMeasurements
            WhichEntries    = WhichEntries[WhichEntries]
            RandomRow       = np.random.randint(0,len(WhichEntries))
            AssocID         = WhichEntries.index.get_level_values(0)[RandomRow]
            Filter          = WhichEntries.index.get_level_values(1)[RandomRow]
        JulianDates = self.phot_df.loc[AssocID, Filter].reset_index().T_Start
        Magnitudes  = self.phot_df.loc[AssocID, Filter].MAG_APER2
        Synth_err   = self.phot_df.loc[AssocID, Filter].SynErrorDevSig
        MAG_err     = self.phot_df.loc[AssocID, Filter].MAGERR_APER2
        
        plt.errorbar(JulianDates, Magnitudes, yerr=Synth_err, linestyle='none', fmt='o', ecolor='black', capthick=2)
        #plt.scatter(JulianDates, Magnitudes, marker='o', c='black')
        

    def median_mad_sigma(self):
        mags, mads = self.GetMedMags(), self.GetMADvalues()  
        mags = pd.DataFrame(mags)
        mags.columns = ['MAG']
        mads = pd.DataFrame(mads)
        mads.columns = ['MAD']
        # Drop first 0.5 mag and last 0.5 mag
        lower_thres = pd.DataFrame(mags.groupby('Filter').min()+0.5)#FilterMags.min()+0.5
        lower_thres.columns = ['LowerMagThresh']
        upper_thres = pd.DataFrame(mags.groupby('Filter').max()-0.5)##FilterMags.max()-0.5
        upper_thres.columns = ['UpperMagThresh']

        mags = pd.merge(mags, lower_thres, left_index=True,left_on='Filter', right_index=True)#, right_on='Filter')
        mags = pd.merge(mags, upper_thres, left_index=True,left_on='Filter', right_index=True)

        mags = mags[(mags.MAG>mags.LowerMagThresh)*(mags.MAG<mags.UpperMagThresh)]
        MagsMads = mags.copy()
        MagsMads = MagsMads.join(mads)

        MagsMads['MagBin'] = pd.cut(MagsMads['MAG'], bins=20)
        MedMadBin = pd.DataFrame(MagsMads.groupby('MagBin')['MAD'].median())
        MedMadBin.columns = ['MedMadBin']
        MagsMads = pd.merge(MagsMads,MedMadBin, left_on='MagBin', right_index=True)

        MagsMads['MADSigma'] = MagsMads['MAD'] / MagsMads['MedMadBin']
        #MagsMads = MagsMads[MagsMads.MADSigma>6]
        return MagsMads
    
    def potential_candidates(self):
        MagsMads = self.median_mad_sigma()
        PotCandidates = MagsMads['MAD'].unstack()
        PotCandidates['NFilters'] = PotCandidates.count(axis=1)
        return PotCandidates

<h3>Code</h3>

In [ ]:
all_stars = Table.read('../../../HST_Guido/30dor_all_newerr.UBVIHa.rot', format='ascii').to_pandas()
all_stars.columns = 'ID;x;y;RA;Dec;u_1;eu_2;b_1;eb_2;v_1;ev_2;i_1;ei_2;ha_1;eha_2'.split(';')
all_stars = all_stars.set_index('ID')
#all_stars = all_stars[np.logical_not(((all_stars.ha_1<-10)+(all_stars.ha_1>50)))]

In [ ]:
#pms_stars = Table.read('../../../HST_Guido/30dorallpmsstars.txt', format='ascii').to_pandas()
#pms_stars = pms_stars.set_index('NR')

In [ ]:
phot_df = read_sexcats().return_func()

In [ ]:
phot_df = HCV_error_analysis(phot_df).clean(MZPcorr=False)

In [ ]:
magnitudes = phot_df.groupby(['AssocID', 'Filter'])['MAG_APER2'].median()
magnitudes = magnitudes.unstack()

mads = Analyzer.GetMADvalues().unstack()

In [ ]:
magnitudes['V_Ha'] = magnitudes['F555W'] - magnitudes['F656N']
magnitudes['V_I'] = magnitudes['F555W'] - magnitudes['F814W']

In [ ]:
phot_df.groupby('Filter')['MAG_APER2'].nunique()

In [ ]:
plt.scatter(magnitudes['V_I'], magnitudes['F555W'], s=1, marker='x', color='grey')
plt.scatter(magnitudes['V_I'].loc[pot_cand], magnitudes['F555W'].loc[pot_cand], s=1, marker='x', color='red')
#plt.scatter(magnitudes['V_I'].loc[pot_cand], magnitudes['V_Ha'].loc[pot_cand], s=1, marker='x', color='red')
plt.ylim(30,10)
plt.xlim(-0.5,3)
plt.ylabel('V')
plt.xlabel('V-I')
plt.show()